In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id, current_timestamp, lit

In [1]:
# Replace with your credentials
aws_access_key_id = "admin"
aws_secret_access_key = "password"

In [16]:
# Create Spark Session
spark = (
    SparkSession.builder.appName("SparkSQL_in_Jupyter")
    .config("spark.jars.packages",
            "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.0,"
            "software.amazon.awssdk:bundle:2.24.8,"
            "software.amazon.awssdk:url-connection-client:2.24.8,"
            "org.apache.hadoop:hadoop-aws:3.3.4,"
            "com.amazonaws:aws-java-sdk-bundle:1.12.375")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.catalog.lakehouse_prod_catalog", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.lakehouse_prod_catalog.type", "nessie")
    .config("spark.sql.catalog.lakehouse_prod_catalog.uri", "http://nessie:19120/api/v1")
    .config("spark.sql.catalog.lakehouse_prod_catalog.warehouse", "s3a://warehouse/")
    .config("spark.sql.catalog.lakehouse_prod_catalog.ref", "main")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", aws_access_key_id)
    .config("spark.hadoop.fs.s3a.secret.key", aws_secret_access_key)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider",
            "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .getOrCreate()
)


25/10/14 16:28:47 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
# Show the data
df = spark.sql("SELECT * FROM lakehouse_prod_catalog.raw_bronze_zone.bronze_hris_employee;")
df.show()

25/10/14 15:58:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

+----------+-----------+--------+---------+---------+--------------------+-----------------+--------------------+------------+--------------+------------+-------+--------------------------+---------------+----------------------+-----------------+--------------------+----------+-----+----------------------+----------+------------+--------+-----------+-----------------+-----------------------+-----------+-----------+----------------+------------------+-----------------------+-------------+---------------------+-------------+----------------+-----------------+--------------------+-----------------------+-------------+-------------------+
|Unnamed: 0|  FirstName|LastName|StartDate| ExitDate|               Title|       Supervisor|             ADEmail|BusinessUnit|EmployeeStatus|EmployeeType|PayZone|EmployeeClassificationType|TerminationType|TerminationDescription|   DepartmentType|            Division|       DOB|State|JobFunctionDescription|GenderCode|LocationCode|RaceDesc|MaritalDesc|Perfo

In [9]:
# Show Catalog (Nessie)
catalog = 'lakehouse_prod_catalog'
spark.sql(f"SHOW NAMESPACES IN {catalog}").show()

+---------------+
|      namespace|
+---------------+
|raw_bronze_zone|
+---------------+



In [19]:
# Read Spark DataFrame
df_spark = spark.table("lakehouse_prod_catalog.raw_bronze_zone.bronze_hris_employee")
df_spark.show()

+----------+-----------+--------+---------+---------+--------------------+-----------------+--------------------+------------+--------------+------------+-------+--------------------------+---------------+----------------------+-----------------+--------------------+----------+-----+----------------------+----------+------------+--------+-----------+-----------------+-----------------------+-----------+-----------+----------------+------------------+-----------------------+-------------+---------------------+-------------+----------------+-----------------+--------------------+-----------------------+-------------+-------------------+
|Unnamed: 0|  FirstName|LastName|StartDate| ExitDate|               Title|       Supervisor|             ADEmail|BusinessUnit|EmployeeStatus|EmployeeType|PayZone|EmployeeClassificationType|TerminationType|TerminationDescription|   DepartmentType|            Division|       DOB|State|JobFunctionDescription|GenderCode|LocationCode|RaceDesc|MaritalDesc|Perfo

In [22]:
df_spark.groupBy("EmployeeType").count().show()

+------------+-----+
|EmployeeType|count|
+------------+-----+
|    Contract| 1055|
|   Full-Time| 1086|
|   Part-Time| 1009|
+------------+-----+



In [23]:
df_spark.createOrReplaceTempView("temp_vw_hris")

In [24]:
spark.sql("""
SELECT *
FROM temp_vw_hris
""").show()

+----------+-----------+--------+---------+---------+--------------------+-----------------+--------------------+------------+--------------+------------+-------+--------------------------+---------------+----------------------+-----------------+--------------------+----------+-----+----------------------+----------+------------+--------+-----------+-----------------+-----------------------+-----------+-----------+----------------+------------------+-----------------------+-------------+---------------------+-------------+----------------+-----------------+--------------------+-----------------------+-------------+-------------------+
|Unnamed: 0|  FirstName|LastName|StartDate| ExitDate|               Title|       Supervisor|             ADEmail|BusinessUnit|EmployeeStatus|EmployeeType|PayZone|EmployeeClassificationType|TerminationType|TerminationDescription|   DepartmentType|            Division|       DOB|State|JobFunctionDescription|GenderCode|LocationCode|RaceDesc|MaritalDesc|Perfo

In [27]:
df_maritial = df_spark.select("MaritalDesc").distinct()

df_maritial_created = df_maritial.withColumn("MaritalDesc")

+-----------+
|MaritalDesc|
+-----------+
|    Married|
|   Divorced|
|    Widowed|
|     Single|
+-----------+



In [30]:
df_drop_dup = df_spark.dropDuplicates()
df_drop_dup.show()

Py4JJavaError: An error occurred while calling o207.showString.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:481)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.comma

In [29]:
spark.stop()